In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#



# A Look at Some Broadband 80 Meter Antenna Ideas

It's well known that a standard dipole antenna can't provide a low VSWR over the entire frequency range of the 80 meter band. This notebook looks at some published designs that improve on the dipole's bandwidth. Some of the code presented in previous notebooks is used to optimize antenna geometries and feedline matching arrangements to extract maximum frequency coverage from the designs.

Note that I haven't built or tested any of these antennas - this is merely a demonstration of simulation and optimization.

Thanks to the authors of the various articles referenced herein.



## Baseline for Comparison: A Simple Dipole




![Graph](../Cad/Series_Section_Match_1.svg)



<!-- <img src="../Cad/Series_Section_Match_1.svg" alt="Drawing" style="width: 400px;"/> -->
<img src="../Cad/Series_Section_Match_1.svg" alt="Drawing" />



![Graph](../Cad/Series_Section_Match_1.png)